In [1]:
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 
import numpy as np
import random
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

In [2]:
Box(0,255,shape=(3,3), dtype=int).sample()

array([[ 53, 141, 231],
       [ 95,  21,  44],
       [  9, 171, 159]])

In [3]:
class ChargeEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Peice array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start SOC
        self.state = 20 + random.randint(-3,3)
        # Set max SOC
        self.max_state = 100
        # Set power
        self.power = 20
        # Set price data
        self.priceArray = np.float32(np.array([36.22, 27.98, 4.6, 9.38, -0.43, 2.26, 4.02, 8.48, 18.73,
                                               20.64, 22.99, 18.93, 16.67, 9.51, 3.59, 2.31, 2.78, 6.3, 14.18,
                                               16.11, 21.42, 25.87, 30.71, 18.44]))
        # Set current index
        self.currentIndex = 0
        self.currentprice = 0
        # Set temp 
        self.temp = 0
        self.change = 0
        # Set median
        self.mid = np.median(self.priceArray)
        self.open = 0
        # Set total time 
        self.time = 23 
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 sell
        # 1 -1 = 0 hold
        # 2 -1 = 1 buy 
        # Update the state
        self.state += (action - 1) * self.power  
        
        self.currentprice = self.priceArray[self.currentIndex]
        
        # Reduce time by 1 h
        self.time -= 1 
        
        # If state exceeds boundaries after the action due to power limit, set state to 100 or 0        
        if 100 < self.state < (self.max_state + self.power):
            self.state = 100
        if (0 - self.power) < self.state < 0:
            self.state = 0
            
        # Calculate reward    
        # BUy
       # Give plus reward when charge at low proce, and minus at high price, and 0 at hold
        if self.currentprice < (self.mid - 2):
            temp = abs(self.currentprice) * (action - 1)
            reward = temp
            
        # Sell
        # Give minus reward when discharge at low proce, and plus at high price, and 0 at hold
        elif self.currentprice > (self.mid + 2):
            temp = -abs(self.currentprice) * (action - 1)
            reward = temp
            
        # Hold
        # Give plus reward when hold in median price range, and 0 reward otherwise
        else:
            temp = abs(abs(action) - 1) * self.mid
            reward = temp
        
        # Move to the next price
        self.currentIndex += 1
        
        # Set Low boundary and Up boundary
        #Set end state
        if self.time <= 0 and self.state == 100: 
            done = True
            reward = 20
        elif self.time <= 0 and self.state >= 90:
            done = True
            reward = 10
        elif self.time <= 0 and self.state >= 80:
            done = True
            reward = 5
        elif self.time <= 0 and self.state < 80:
            done = True
            reward = -1000
        elif self.state < 0 or self.state > 100:
            done = True
            reward = -1000
        else:
            done = False       

        

        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = np.array([20 + random.randint(-3,3)])#.astype(float)
        # Set max SOC
        self.max_state = 100
        # Set power
        self.power = 20
        # Set price data
        self.priceArray = np.float32(np.array([36.22, 27.98, 4.6, 9.38, -0.43, 2.26, 4.02, 8.48, 18.73,
                                               20.64, 22.99, 18.93, 16.67, 9.51, 3.59, 2.31, 2.78, 6.3, 14.18,
                                               16.11, 21.42, 25.87, 30.71, 18.44]))
        # Set current index
        self.currentIndex = 0
        self.currentprice = 0
        # Set temp 
        self.temp = 0
        self.change = 0
        # Set median
        self.mid = np.median(self.priceArray)
        self.open = 0
        # Set total time 
        self.time = 23 
        return self.state

In [4]:
env=ChargeEnv()

C:\Users\Yuche\AppData\Roaming\Python\Python39\site-packages\gym\spaces\box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [5]:
env.observation_space.sample()

array([43.63738], dtype=float32)

In [6]:
env.reset()

array([17])

In [7]:
from stable_baselines3.common.env_checker import check_env

In [8]:
check_env(env, warn=True)

AssertionError: The observation returned by the `reset()` method does not match the given observation space

In [9]:
episodes = 50
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{} State:{}'.format(episode, score, n_state))
env.close()

Episode:1 Score:-993.869998395443 State:-20
Episode:2 Score:-961.9999980330467 State:[40]
Episode:3 Score:-962.7599990367889 State:120
Episode:4 Score:-1013.2700015902519 State:-20
Episode:5 Score:-1001.1399984359741 State:-20
Episode:6 Score:46.2499982714653 State:[80]
Episode:7 Score:-1048.3900007605553 State:120
Episode:8 Score:-907.1150005459785 State:40
Episode:9 Score:-999.9400015473366 State:-20
Episode:10 Score:-1011.5500022768974 State:-20
Episode:11 Score:-963.7799987792969 State:[-20]
Episode:12 Score:-963.7799987792969 State:-20
Episode:13 Score:-985.0699982047081 State:-20
Episode:14 Score:-1008.320000231266 State:120
Episode:15 Score:-1021.5500002503395 State:120
Episode:16 Score:-1005.659999191761 State:40
Episode:17 Score:-1041.8950009942055 State:[40]
Episode:18 Score:-935.7999992370605 State:-20
Episode:19 Score:-1037.7899996638298 State:[-20]
Episode:20 Score:-1072.470000743866 State:120
Episode:21 Score:-1012.210001707077 State:[120]
Episode:22 Score:-1016.234997749

In [10]:
log_path = os.path.join('Training', 'Logs')

In [11]:
model = PPO("MlpPolicy", env, learning_rate=0.0001, verbose=1, tensorboard_log=log_path)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [12]:
model.learn(total_timesteps=500000)

Logging to Training\Logs\PPO_27
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 12.7     |
|    ep_rew_mean     | -915     |
| time/              |          |
|    fps             | 302      |
|    iterations      | 1        |
|    time_elapsed    | 6        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 15           |
|    ep_rew_mean          | -895         |
| time/                   |              |
|    fps                  | 326          |
|    iterations           | 2            |
|    time_elapsed         | 12           |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0093871355 |
|    clip_fraction        | 0.0731       |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.09        |
|    explained_variance   

KeyboardInterrupt: 

In [13]:
#evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [14]:
#model.save('PPO')

In [42]:
episodes = 100
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action, _states = model.predict(obs)
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{} State:{}'.format(episode, score, n_state))
env.close()

Episode:1 Score:-49.61000108718872 State:[82]
Episode:2 Score:-49.820000648498535 State:[82]
Episode:3 Score:10.369999885559082 State:100
Episode:4 Score:-8.189999103546143 State:100
Episode:5 Score:-15.640000104904175 State:[100]
Episode:6 Score:-214.33999919891357 State:[60]
Episode:7 Score:-206.2300000190735 State:[79]
Episode:8 Score:29.359999418258667 State:[98]
Episode:9 Score:-262.35999870300293 State:[79]
Episode:10 Score:49.555000483989716 State:100
Episode:11 Score:-28.760001182556152 State:100
Episode:12 Score:13.660000085830688 State:100
Episode:13 Score:-9.789999723434448 State:[98]
Episode:14 Score:-194.10000014305115 State:[57]
Episode:15 Score:-200.0 State:[20]
Episode:16 Score:-6.219999551773071 State:[81]
Episode:17 Score:-227.660001039505 State:[78]
Episode:18 Score:-23.199999809265137 State:[98]
Episode:19 Score:21.305000066757202 State:100
Episode:20 Score:3.720000207424164 State:[100]
Episode:21 Score:-20.339999496936798 State:[99]
Episode:22 Score:-1059.170000851

In [ ]:
env.close()

In [ ]:
#Save the model
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_model')

In [ ]:
model.save(PPO_path)

In [ ]:
del model

In [ ]:
model = PPO.load('PPO_model', env=env)

In [ ]:
#View logs in Tensorboard
training_log_path = os.path.join(log_path, 'PPO')
!tensorboard --logdir={training_log_path}